# Data integration - OpenCivitas

In questo IPython Notebook è presente la procedura utilizzata per integrare i dati OpenCivitas con info dei comuni. L'output della procedura sono una serie di file, che vedono l'aggiunta di alcune variabili rispetto i dati iniziali. Nel complesso, l'informazione contenuta nei file è la seguente:


| Nome | Tipo   | Descrizione                     |
|------|--------|---------------------------------|
| ANNO | number | Anno di analisi  |
| Codice_Nazionale | string  | codice catastale del comune |
| Codice_Istat | string  | codice ISTAT |
| Denominazione Italiana | string  | Nome comune |
| Provincia | string  | Nome capoluogo di provincia cui appartiene il comune |
| Sigla Provincia | string  | Sigla capoluogo di provincia cui appartiene il comune |
| Codice_Provincia | int  | Codice capoluogo di provincia cui appartiene il comune |
| Regione | string  | Regione cui appartiene il comune |
| Sigla Regione | string  | Sigla regione cui appartiene il comune |
| Codice_Regione | int  | Codice regione cui appartiene il comune |
| AreaGeo | string  | Area geografica cui appartiene il comune |
| ... | ... | ...|
| ... | ... | ...|
| ... | ... | ...|
| __Variabili in fabbisogni__|||
| ... | ... | ...|
| ... | ... | ...|
| ... | ... | ...|

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os

* Carichiamo la lista dei fali che dobbiamo modificare

In [2]:
# Importare lista dati
lista_file = os.listdir("data/")

* Per ogni file compiamo le operazioni descritte in seguito:

__A seguire__: una descrizione più dei risultati intermedi ad ogni passaggio per un unico dataset su un anno specifico.

# Esempio con uno specifico dataset di fabbisogni e anno

In [3]:
df_fab = pd.read_csv('data/FC03U_funz_istruzione.csv', sep = ';')
anno = 2013

* Estraiamo le righe del dataset corrispondenti all'anno

In [4]:
df_fab_anno = df_fab[df_fab["ANNO"] == anno]
df_fab_anno.head(3)

,ANNO,COMUNE_CAT_COD,CAT_DET_COD_1,CAT_DET_COD_2,CAT_DET_COD_3,CAT_DET_COD_4,CAT_DET_COD_5,COEFF_TEORICO,IND1,IND3,...,IND8,IND9,IND10,LQP_COD_1,LQP_COD_2,LQP_COD_3,LQP_COD_4,LQP_COD_5,LQP_COD_6,LQP_COD_S
6702,2013,A052,0.353504,0.005258,0.615990,0.022052,0.003197,0.000197,39.952075,12.081945,...,8.834500,6.284809,0.0,12.081945,-32.161687,0,0,7.0,3.0,3.4
6703,2013,A146,0.101410,0.000000,0.858842,0.000000,0.039748,0.000004,86.529052,110.956579,...,NaN,NaN,NaN,110.956579,-61.790313,0,3,10.0,1.0,1.0
6704,2013,A182,0.423633,0.003823,0.523610,0.045140,0.003794,0.002068,56.140196,-30.230513,...,5.791911,8.196974,0.0,-30.230513,15.214745,0,0,3.0,7.0,7.4


In [6]:
print ('Numero comuni anno '+ str(anno) +':', len(df_fab_anno['COMUNE_CAT_COD'].unique()))

Numero comuni anno 2013: 6699


* Carichiamo il dataset che contiene l'elenco di tutti comuni __MODIFICA: utilizzo altro dataset__

In [7]:
# Carico il file 
#df_comuni = pd.read_csv('ElencoComuniAttuali_20170918.csv', sep = ';')
# Trattengo le variabili di interesse
#df_comuni = df_comuni[['Codice Nazionale','Codice Istat','Denominazione Italiana', 'Sigla Provincia']] 
# E le rinomino per poter fare la join con il dataset dei fabbisogni
#df_comuni.columns = ['COMUNE_CAT_COD','ISTAT','Denominazione Italiana', 'Sigla Provincia']


# Carico il file 
df_comuni = pd.read_csv('listacomuni.txt', sep = ';', encoding='cp1250')

# E le rinomino per poter fare la join con il dataset dei fabbisogni
df_comuni.columns = ['Istat', 'Comune', 'Provincia', 'Regione', 'Prefisso', 'CAP',
       'COMUNE_CAT_COD', 'Abitanti', 'Link']

# Trattengo le variabili di interesse
df_comuni = df_comuni[['Istat', 'Comune', 'Provincia', 'Regione', 'Prefisso', 'CAP',
       'COMUNE_CAT_COD', 'Abitanti']] 

In [8]:
df_comuni.head(3)

,Istat,Comune,Provincia,Regione,Prefisso,CAP,COMUNE_CAT_COD,Abitanti
0,28001,Abano Terme,PD,VEN,49,35031,A001,19726
1,98001,Abbadia Cerreto,LO,LOM,371,26834,A004,293
2,97001,Abbadia Lariana,LC,LOM,341,23821,A005,3305


In [10]:
print ("Numero comuni: ",len(set(df_fab_anno['COMUNE_CAT_COD'])))

print ("Numero comuni cui dati possono essere integrati: ", len(set(df_fab_anno['COMUNE_CAT_COD']).intersection(set(df_comuni['COMUNE_CAT_COD']))))

print ("Comuni mancanti: ", set(df_fab_anno['COMUNE_CAT_COD']) - set(df_comuni['COMUNE_CAT_COD']))

Numero comuni:  6699
Numero comuni cui dati possono essere integrati:  6699
Comuni mancanti:  set()


__SANITY CHECK__: Il join va bene se l'uguaglianza è vera

In [11]:
len(set(df_fab_anno['COMUNE_CAT_COD'])) == len(set(df_fab_anno['COMUNE_CAT_COD']).intersection(set(df_comuni['COMUNE_CAT_COD'])))

True

Il sanity chech rileva che le informazioni relative a tre comuni non sono state integrati perchè non presenti nel dataset che abbiamo utilizzato per fare la join.

Visto il piccolo numero di comuni mancanti, non presenti nemmeno del dataset `ElencoComuniAttuali_20170918.csv`, si è cercato su Google. Tutti e tre i comuni risultano essere stati __soppressi__ nel 2011, i comuni non compaiono nei due dataset poichè entrambi più aggiornati del 2010.

* E151: [Gravedone](http://www.tuttitalia.it/lombardia/86-gravedona/)
* D986: [Germasino](http://www.tuttitalia.it/lombardia/41-germasino/)
* C965: [Consiglio di Rumo](http://www.tuttitalia.it/lombardia/60-consiglio-di-rumo/)

I tre sono stati accorpati in un unico comune. I comuni che non sono associati a codice ISTAT __non__ sono esclusi dal dataset.

__Attenzione:__ queste considerazioni valgono esclusivamente per i dati del 2010

* Facciamo la join tra il dataset dei fabbisogni e quello dei comuni

In [12]:
# Left join tra i fabbisogni ed i dati dei comuni

#df_comuni_cod = pd.merge(df_fab_anno,df_comuni, on = 'COMUNE_CAT_COD',how = "left")
df_comuni_cod = pd.merge(df_fab_anno,df_comuni, on = 'COMUNE_CAT_COD',how = "left")

df_comuni_cod = df_comuni_cod[['ANNO', 'COMUNE_CAT_COD', 'Istat', 'Comune', 'Provincia',  'Regione']]

df_comuni_cod.columns = ['ANNO', 'COMUNE_CAT_COD', 'ISTAT', 'Comune', 'Provincia',  'Regione']

In [14]:
df_comuni_cod.head(3)

,ANNO,COMUNE_CAT_COD,ISTAT,Comune,Provincia,Regione
0,2013,A052,6001,Acqui Terme,AL,PIE
1,2013,A146,6002,Albera Ligure,AL,PIE
2,2013,A182,6003,Alessandria,AL,PIE


* Carico il seguente dataset ISTAT

In [15]:
# Carica il file che racchiude informazioni extra sui comuni (provincia, regione, area geografica)
df_comuni_info = pd.read_csv('info_comuni.txt', sep = ";")
df_comuni_info.head(3)

,Comune,ISTAT,Provincia,SiglaProv,Regione,AreaGeo,PopResidente,PopStraniera,DensitaDemografica,SuperficieKmq,...,AltezzaMassima,ZonaAltimetrica,TipoComune,GradoUrbaniz,IndiceMontanita,ZonaClimatica,ZonaSismica,ClasseComune,Latitudine,Longitudine
0,Abano Terme,28001,Padova,PD,Veneto,Nord-Est,20002,2001,"931,89","21,408",...,80,Montagna Interna,No capoluogo,Elevato,Non montano,E,4,Polo di attrazione intercomunale,"45,3594444444444","11,7894444444444"
1,Abbadia Cerreto,98001,Lodi,LO,Lombardia,Nord-Ovest,284,13,"46,62","6,199",...,70,Pianura,No capoluogo,Medio,Non montano,E,4,Area di cintura,"45,3122222222222","9,59277777777778"
2,Abbadia Lariana,97001,Lecco,LC,Lombardia,Nord-Ovest,3209,135,"191,93","16,673",...,1700,Montagna Interna,No capoluogo,Medio,Totalmente montano,E,4,Area periferica,"45,8991666666667","9,33361111111111"


Anche in questo caso, probabilmente a causa dell'aggiornamento delle tabelle ISTAT dopo la doppressione dei comuni, troviamo che 402 dei comuni in OpenCivitas non hanno informazioni nel dataset `info_comuni`.

In [16]:
print ("Information not in the dataset: ", 6999 - len(set(df_comuni_cod.ISTAT).intersection(set(df_comuni_info.ISTAT))))

Information not in the dataset:  420


Pertanto proviamo ad ottenere le informazioni mancanti tramite altri dataset:

In [17]:
df_comuni_info_2 = pd.read_excel('Elenco_comuni_italiani_1¯_gennaio_2010.xls', sheetname="COMUNI 01_01_2010")

* Seleziona variabili di interesse

In [18]:
df_comuni_info_2 = df_comuni_info_2[['Codice Istat del Comune \n(formato numerico)','Codice Regione', 'Codice Provincia']]
df_comuni_info_2.columns = ['ISTAT', 'Codice_Regione', 'Codice_Provincia']

df_comuni_info_2.head(3)

,ISTAT,Codice_Regione,Codice_Provincia
0,1001,1,1
1,1002,1,1
2,1003,1,1


* Utilizziamo il seguente dataset di ISTAT per collegare i codici di regione e provincia ai nomi effettivi.

In [19]:
df_regioni_province = pd.read_excel('ripartizioni_regioni_province.xls', sheetname= "COD_REG_PROV")

In [20]:
df_regioni_province.columns = ['Codice ripartizione', 'Codice NUTS1',
       'Ripartizione geografica\n(Maiuscolo)', 'Ripartizione geografica',
       'Codice_Regione', 'Codice NUTS2 2006(a)',
       'Denominazione regione\n(Maisucolo)', 'Denominazione regione',
       'Codice_Provincia', 'Codice NUTS3 2006', 'Denominazione provincia',
       'Sigla automobilistica']

In [21]:
df_regioni_province.head(3)

,Codice ripartizione,Codice NUTS1,Ripartizione geografica (Maiuscolo),Ripartizione geografica,Codice_Regione,Codice NUTS2 2006(a),Denominazione regione (Maisucolo),Denominazione regione,Codice_Provincia,Codice NUTS3 2006,Denominazione provincia,Sigla automobilistica
0,1,ITC,NORD-OVEST,Nord-ovest,1.0,ITC1,PIEMONTE,Piemonte,1.0,ITC11,Torino,TO
1,1,ITC,NORD-OVEST,Nord-ovest,1.0,ITC1,PIEMONTE,Piemonte,2.0,ITC12,Vercelli,VC
2,1,ITC,NORD-OVEST,Nord-ovest,1.0,ITC1,PIEMONTE,Piemonte,3.0,ITC15,Novara,NO


* Facciamo la join tra i due dataset

In [22]:
data_regioni = pd.merge(df_regioni_province, df_comuni_info_2, on = ['Codice_Regione','Codice_Provincia'],how = "outer")

data_regioni = data_regioni[['ISTAT',
       'Codice_Regione', 'Denominazione regione',
       'Codice_Provincia', 'Denominazione provincia','Ripartizione geografica'
        ]]

* La variabile `ISTAT` corrisponde al codice ISTAT del comune.

In [24]:
data_regioni.head(3)

,ISTAT,Codice_Regione,Denominazione regione,Codice_Provincia,Denominazione provincia,Ripartizione geografica
0,1001.0,1,Piemonte,1,Torino,Nord-ovest
1,1002.0,1,Piemonte,1,Torino,Nord-ovest
2,1003.0,1,Piemonte,1,Torino,Nord-ovest


* Join tra `df_comuni_cod` e `data_regioni`

In [62]:
df_info_comuni_fabbisogni = pd.merge(df_comuni_cod, data_regioni, on = 'ISTAT',how = 'left')
df_info_comuni_fabbisogni = df_info_comuni_fabbisogni[['ANNO', 'COMUNE_CAT_COD', 'ISTAT', 'Comune',
                                                         'Denominazione provincia', 'Provincia', 'Codice_Provincia', 
                                                         'Denominazione regione', 'Regione', 
                                                       'Codice_Regione', 'Ripartizione geografica']]

In [63]:
df_info_comuni_fabbisogni.head(3)

,ANNO,COMUNE_CAT_COD,ISTAT,Comune,Denominazione provincia,Provincia,Codice_Provincia,Denominazione regione,Regione,Codice_Regione,Ripartizione geografica
0,2013,A052,6001,Acqui Terme,Alessandria,AL,6,Piemonte,PIE,1,Nord-ovest
1,2013,A146,6002,Albera Ligure,Alessandria,AL,6,Piemonte,PIE,1,Nord-ovest
2,2013,A182,6003,Alessandria,Alessandria,AL,6,Piemonte,PIE,1,Nord-ovest


* Si nota che i per la provincia di Napoli non si ha la sigla della provincia. Pertanto si provvede ad inserirla. (Questa operazione è stata spostata a fine pipeline per evitare bug mentre si rinominano le colonne)

In [64]:
#df_info_comuni_fabbisogni = df_info_comuni_fabbisogni.fillna({'Provincia':'NA'})

* Infine, prima di salvare il dataset aggiungiamo le variabili mancanti e le rinominiamo

In [65]:
# Resetta gli indici per concatenare bene
dat1 = df_info_comuni_fabbisogni.reset_index(drop=True)
dat2 = df_fab_anno.iloc[:,2:].reset_index(drop=True)
df_info_comuni_fabbisogni = pd.concat([dat1,dat2], axis=1)

* Define the new list of columns

In [29]:
vecchie_colonne = df_fab_anno.iloc[:,2:].columns

In [30]:
final_colonne = ['Anno', 'Codice_Nazionale', 'Codice_Istat', 'Denominazione Italiana', 'Provincia', 'Sigla Provincia','Codice_Provincia',
       'Regione', 'Sigla Regione', 'Codice_Regione','AreaGeo']+list(vecchie_colonne)

In [31]:
# Rinomina variabili
df_info_comuni_fabbisogni.columns = final_colonne

In [32]:
df_info_comuni_fabbisogni.head()

,Anno,Codice_Nazionale,Codice_Istat,Denominazione Italiana,Provincia,Sigla Provincia,Codice_Provincia,Regione,Sigla Regione,Codice_Regione,...,IND8,IND9,IND10,LQP_COD_1,LQP_COD_2,LQP_COD_3,LQP_COD_4,LQP_COD_5,LQP_COD_6,LQP_COD_S
0,2013.0,A052,6001,Acqui Terme,Alessandria,AL,6,Piemonte,PIE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013.0,A146,6002,Albera Ligure,Alessandria,AL,6,Piemonte,PIE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013.0,A182,6003,Alessandria,Alessandria,AL,6,Piemonte,PIE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013.0,A189,6004,Alfiano Natta,Alessandria,AL,6,Piemonte,PIE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013.0,A197,6005,Alice Bel Colle,Alessandria,AL,6,Piemonte,PIE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Inserimento sigla della provincia di Napoli.

In [33]:
df_info_comuni_fabbisogni = df_info_comuni_fabbisogni.fillna({'Sigla Provincia':'NA'})

* Quindi salviamo il dataset in un csv.

La procedura appena descritta è automatizzatata con la seguente funzione e ripetuta per tutti i file relativi ai fabbisogni.

In [3]:
def HandleData(file_name, comuni_attuali = 'Elenco_comuni_italiani_1¯_gennaio_2010.xls' ,info_comuni = 'ripartizioni_regioni_province.xls'):
    """ La funzione crea un nuovo dataset .csv file.
    @file_name: nome del file di fabbisogni
    @comuni_attuali: file 'Elenco_comuni_italiani_1¯_gennaio_2010.xls che contiene le info ed i codici per i comuni
    @info_comuni: file ripartizioni_regioni_province.xls che contiene  info su regioni e province"""
    
    df_fab = pd.read_csv('data/'+file_name, sep = ';')
    
    # Per ogni anno trovato nel dataset
    for anno in df_fab.ANNO.unique():
        
        # Estrazione righe dataset di interesse
        df_fab_anno = df_fab[df_fab["ANNO"] == anno]
        print ('Numero comuni anno '+ str(anno) +':', len(df_fab_anno['COMUNE_CAT_COD'].unique()))


        # Carico il file 
        df_comuni = pd.read_csv('listacomuni.txt', sep = ';', encoding='cp1250')

        # E le rinomino per poter fare la join con il dataset dei fabbisogni
        df_comuni.columns = ['Istat', 'Comune', 'Provincia', 'Regione', 'Prefisso', 'CAP',
               'COMUNE_CAT_COD', 'Abitanti', 'Link']

        # Trattengo le variabili di interesse
        df_comuni = df_comuni[['Istat', 'Comune', 'Provincia', 'Regione', 'Prefisso', 'CAP',
               'COMUNE_CAT_COD', 'Abitanti']] 

        print ("SANITY CHECK 1: ")
        print ("Numero comuni: ",len(set(df_fab_anno['COMUNE_CAT_COD'])))

        print ("Numero comuni cui dati possono essere integrati: ", len(set(df_fab_anno['COMUNE_CAT_COD']).intersection(set(df_comuni['COMUNE_CAT_COD']))))

        print ("Comuni mancanti: ", set(df_fab_anno['COMUNE_CAT_COD']) - set(df_comuni['COMUNE_CAT_COD']))
        
        
        # Left join tra i fabbisogni ed i dati dei comuni
        df_comuni_cod = pd.merge(df_fab_anno,df_comuni, on = 'COMUNE_CAT_COD',how = "left")
        
        # Seleziona variabili
        df_comuni_cod = df_comuni_cod[['ANNO', 'COMUNE_CAT_COD', 'Istat', 'Comune', 'Provincia',  'Regione']]
        
        # Rinomina Istat
        df_comuni_cod.columns = ['ANNO', 'COMUNE_CAT_COD', 'ISTAT', 'Comune', 'Provincia',  'Regione']
        
        print ('SANITY CHECK 1: Numero di comuni dopo la join: ', len(df_comuni_cod.COMUNE_CAT_COD.unique()))
        
        
        # Carica il file che racchiude informazioni extra sui comuni (provincia, regione, area geografica)
        df_comuni_info_2 = pd.read_excel(comuni_attuali, sheetname="COMUNI 01_01_2010")
        # Seleziona e rinomina variabili di intreresse
        df_comuni_info_2 = df_comuni_info_2[['Codice Istat del Comune \n(formato numerico)','Codice Regione', 'Codice Provincia']]
        df_comuni_info_2.columns = ['ISTAT', 'Codice_Regione', 'Codice_Provincia']
        
        df_regioni_province = pd.read_excel(info_comuni, sheetname= "COD_REG_PROV")
        df_regioni_province.columns = ['Codice ripartizione', 'Codice NUTS1',
       'Ripartizione geografica\n(Maiuscolo)', 'Ripartizione geografica',
       'Codice_Regione', 'Codice NUTS2 2006(a)',
       'Denominazione regione\n(Maisucolo)', 'Denominazione regione',
       'Codice_Provincia', 'Codice NUTS3 2006', 'Denominazione provincia',
       'Sigla automobilistica']
        
        
        data_regioni = pd.merge(df_regioni_province, df_comuni_info_2, on = ['Codice_Regione','Codice_Provincia'],how = "outer")

        data_regioni = data_regioni[['ISTAT',
               'Codice_Regione', 'Denominazione regione',
               'Codice_Provincia', 'Denominazione provincia','Ripartizione geografica'
                ]]
        
        
        df_info_comuni_fabbisogni = pd.merge(df_comuni_cod, data_regioni, on = 'ISTAT',how = 'left')
        df_info_comuni_fabbisogni = df_info_comuni_fabbisogni[['ANNO', 'COMUNE_CAT_COD', 'ISTAT', 'Comune',
                                                         'Denominazione provincia', 'Provincia', 'Codice_Provincia', 
                                                         'Denominazione regione', 'Regione', 
                                                               'Codice_Regione', 'Ripartizione geografica']]
        
        #df_info_comuni_fabbisogni = df_info_comuni_fabbisogni.fillna({'Provincia':'NA'})#])'NA', axis = 1, inplace = True)
        
        
        # Add fabbisogni columns to the df
        # Resetta gli indici per concatenare bene
        dat1 = df_info_comuni_fabbisogni.reset_index(drop=True)
        dat2 = df_fab_anno.iloc[:,2:].reset_index(drop=True)
        df_info_comuni_fabbisogni = pd.concat([dat1,dat2], axis=1)
        
        # Crete new vector of colums to rename
        vecchie_colonne = df_fab_anno.iloc[:,2:].columns
        final_colonne = ['Anno', 'Codice_Nazionale', 'Codice_Istat', 
                         'Denominazione Italiana', 'Provincia', 
                         'Sigla Provincia','Codice_Provincia',
       'Regione', 'Sigla Regione', 'Codice_Regione','AreaGeo']+list(vecchie_colonne)
        
        # Rinomina variabili dataset finale
        df_info_comuni_fabbisogni.columns = final_colonne
        
        df_info_comuni_fabbisogni = df_info_comuni_fabbisogni.fillna({'Sigla Provincia':'NA'})
        
        
        print ('*'*30)

        print (df_info_comuni_fabbisogni[df_info_comuni_fabbisogni['Sigla Provincia'].isnull() == True])
        print (len(df_info_comuni_fabbisogni))
        df_info_comuni_fabbisogni.to_csv(file_name[:-4] +str(anno)+ '.csv', sep = ',')

In [4]:
for file_name in lista_file:
    HandleData(file_name)

Numero comuni anno 2010: 6702
SANITY CHECK 1: 
Numero comuni:  6702
Numero comuni cui dati possono essere integrati:  6699
Comuni mancanti:  {'D986', 'C965', 'E151'}
SANITY CHECK 1: Numero di comuni dopo la join:  6702
******************************
Empty DataFrame
Columns: [Anno, Codice_Nazionale, Codice_Istat, Denominazione Italiana, Provincia, Sigla Provincia, Codice_Provincia, Regione, Sigla Regione, Codice_Regione, AreaGeo, CAT_DET_COD_1, CAT_DET_COD_2, CAT_DET_COD_3, CAT_DET_COD_4, CAT_DET_COD_5, COEFF_TEORICO, IND1, IND3, IND4, IND5, IND6, IND7, IND8, IND9, IND10, IND11, LQP_COD_1, LQP_COD_2, LQP_COD_3, LQP_COD_4, LQP_COD_5, LQP_COD_6, LQP_COD_S]
Index: []

[0 rows x 34 columns]
6708
Numero comuni anno 2013: 6699
SANITY CHECK 1: 
Numero comuni:  6699
Numero comuni cui dati possono essere integrati:  6699
Comuni mancanti:  set()
SANITY CHECK 1: Numero di comuni dopo la join:  6699
******************************
Empty DataFrame
Columns: [Anno, Codice_Nazionale, Codice_Istat, Denom